# 正則化 (Regularization)


In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchinfo
from tqdm import tqdm
import utils

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.3の大きさのガウシアンノイズがのったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
# データ点の取得
x, t = utils.dataset_overtraining(n=30, noise_scale=0.3)

# 学習データと検証用データに分割
from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.30, random_state=0)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x_train, t_train, s=10, c="black", label="training data")
ax.scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
x_grid = np.linspace(-np.pi, np.pi, 100)
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

# numpy -> torch.tensor
from torch import from_numpy

x_grid_tensor = from_numpy(x_grid).float().unsqueeze(-1)
x_tensor = from_numpy(x).float().unsqueeze(-1)
t_tensor = from_numpy(t).float().unsqueeze(-1)
x_train_tensor = from_numpy(x_train).float().unsqueeze(-1)
t_train_tensor = from_numpy(t_train).float().unsqueeze(-1)
x_valid_tensor = from_numpy(x_valid).float().unsqueeze(-1)
t_valid_tensor = from_numpy(t_valid).float().unsqueeze(-1)

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、ノード数が128の隠れ層を5層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

## L2 正則化
過学習の問題は、トレーニングサンプルの数が少なすぎる時、もしくはモデルの表現能力が高すぎる、パラメータの自由度が大きすぎると起こります。
校舎を抑制するため、パラメータの自由度を適度に制約してやることで過学習を抑えることができます。

正則化(Regularization)は、パラメータに制限をかけることで、過学習を抑制します。

代表的なものはL2 正則、もしくは *荷重減衰(weight decay)* と呼ばれる手法で、重みパラメータの2乗和
$$
L_{\text L2} = \sum_i w_i^2
$$
の大きさでペナルティーをかけることで、重みパラメータが過度に大きくならないようにします。

(実はこの制約のかけ方は、$w_i$の事前分布としてガウス分布を仮定すると、導出されます。興味のあるかたは調べてみてください。)



PyTorchでは、以下のようにしてL2ロスを実装できます。

```python
l2_lambda = 0.01
l2 = torch.tensor(0., requires_grad=True)
for weights in model.parameters():
    l2 = l2 + weights.square().sum()
loss_reg = loss + l2_lambda * l2
```

`l2_labmda`の大きさを変更することで、ペナルティの大きさを調整することもできます。 

例えば、`l2_labmda = 0.01`のようにした場合、
$$
L_{\text total} = L + 0.01 \cdot \sum_i w_i^2
$$
のようなロス関数を最小化することになります。

実際に学習をさせてみましょう。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
l2_lambda = 0.01
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # L2正則化項の追加
    l2 = torch.tensor(0.0, requires_grad=True)
    for weights in model.parameters():
        l2 = l2 + weights.square().sum()
    loss_reg = loss + l2_lambda * l2

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss_reg.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    # L2を除いたロス関数の推移を取得します
    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

正則化によって過学習が抑制されてます。また検証用データに誤差関数の値が十分に小さく抑えられているはずです。

PyTorchではoptimizerの引数に値を指定することでもL2正則化を入れることができます。L2正則化を荷重減衰(weight decay)と呼ぶこともありますが、optimizerの引数の`weight decay`を使ってL2正則化を適用できます。

```python
optimizer = Adam(model.parameters(), weight_decay=0.01)
```

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.01)

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    # L2を除いたロス関数の推移を取得します
    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

## L1正則化
もう一つよく使われる正則化手法として、L1 正則化があります。
こちらは、Lasso とも呼ばれることがあります。

L1 正則化では、
$$
L_{\text L1} = \sum_i |w_i|
$$
の形で誤差関数にペナルティーを与えます。
L1正則化は、L2正則化と比較して、$w_i$が0になりやすいという特徴があります。
このことから、モデルパラメータを疎にする目的でこの手法が使われることもあります。

PyTorchでもL2正則化と同様にして実装できます。

```python
l1_lambda = 0.001
l1 = tensor(0., requires_grad=True)
for weights in model.parameters():
    l1 = l1 + weights.abs().sum()
loss_reg = loss + l1_lambda * l1
```


In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
l1_lambda = 0.001
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # L1正則化項の追加
    l1 = torch.tensor(0.0, requires_grad=True)
    for weights in model.parameters():
        l1 = l1 + weights.abs().sum()
    loss_reg = loss + l1_lambda * l1

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss_reg.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    # L2を除いたロス関数の推移を取得します
    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

また、L1正則化とL2正則化の両方をかけることもできます。
$$
L_{\text total} = L + \lambda_{L1} \cdot \sum_i |w_i| + \lambda_{L2} \cdot \sum_i w_i^2
$$

```python
l1_labmda = 0.001
l2_labmda = 0.01
l1 = tensor(0., requires_grad=True)
l2 = tensor(0., requires_grad=True)
for weights in model.parameters():
    l1 = l1 + weights.abs().sum()
    l2 = l2 + weights.square().sum()
loss_reg = loss + l1_lambda * l1 + l2_lambda * l2
```

L1, L2正則化の大きさを変化させて、モデルの予測がどのように変化するかを調べてみましょう。